In [5]:
# 프로젝트: 개인 색 분류를 위한 CLIP 제로샷 학습 파이프라인

# 라이브러리 설치 및 임포트
!pip install git+https://github.com/openai/CLIP.git
!pip install torch torchvision scikit-learn

import torch
import clip
from PIL import Image
import os
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np
from google.colab import files

# 데이터 경로 설정 (데이터셋 경로를 colab에서 'content'에 업로드했다고 가정)
DATASET_PATH = "/content/personal-color-dataset"

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-y0gwjqya
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-y0gwjqya
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [6]:
# CLIP 모델 및 전처리기 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 데이터셋 전처리
def load_images_from_dataset(base_folder):
    images = []
    filenames = []
    labels = []
    for dataset_folder in ["personal-color-1", "personalcolor-2"]:
        dataset_path = os.path.join(base_folder, dataset_folder)
        for split in ["train", "valid", "test"]:
            split_path = os.path.join(dataset_path, split)
            if os.path.exists(split_path):
                for season in ["spring", "summer", "fall", "winter"]:
                    season_folder = os.path.join(split_path, season)
                    if os.path.exists(season_folder):
                        for filename in os.listdir(season_folder):
                            if filename.endswith(".jpg") or filename.endswith(".png"):
                                img_path = os.path.join(season_folder, filename)
                                image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
                                images.append(image)
                                filenames.append(f"{dataset_folder}/{split}/{season}/{filename}")
                                # labeling (spring: 0, summer: 1, fall: 2, winter: 3)
                                if "spring" in season:
                                    labels.append(0)  # Spring color type
                                elif "summer" in season:
                                    labels.append(1)  # Summer color type
                                elif "fall" in season:
                                    labels.append(2)  # Fall color type
                                elif "winter" in season:
                                    labels.append(3)  # Winter color type
    return images, filenames, labels

images, filenames, true_labels = load_images_from_dataset(DATASET_PATH)

In [7]:
# 데이터셋이 비어있는지 확인
if len(images) == 0:
    print("Error: No images found in the dataset path.")
else:
    # 텍스트 레이블
    labels = ["a person with a spring color type", "a person with a summer color type", "a person with a fall color type", "a person with a winter color type"]
    text_inputs = clip.tokenize(labels).to(device)

    # 제로샷 학습
    predicted_labels = []
    with torch.no_grad():
        for i, image in enumerate(images):
            # 이미지 특징 추출
            image_features = model.encode_image(image)
            # 텍스트 특징 추출
            text_features = model.encode_text(text_inputs)

            # 이미지와 텍스트 간 유사도 계산
            logits_per_image, logits_per_text = model(image, text_inputs)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            # 예측 레이블 저장
            predicted_label = np.argmax(probs)
            predicted_labels.append(predicted_label)

    # 성능 평가
    if len(true_labels) > 0 and len(predicted_labels) > 0:
        accuracy = accuracy_score(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels, average='weighted')
        conf_matrix = confusion_matrix(true_labels, predicted_labels)

        # accuracy
        evaluation_results = f"Accuracy: {accuracy * 100:.2f}%\nF1 Score: {f1:.2f}\nConfusion Matrix:\n{conf_matrix}\n"
        print(evaluation_results)

        # 추론 결과 저장
        results_path = "results.txt"
        with open(results_path, "w") as f:
            for i, image in enumerate(images):
                f.write(f"Image: {filenames[i]}\n")
                for j, label in enumerate(labels):
                    f.write(f" - {label}: {probs[0][j] * 100:.2f}%\n")
                f.write(f"Predicted Label: {labels[predicted_labels[i]]}\n")
                f.write(f"True Label: {labels[true_labels[i]]}\n")
                f.write("\n")

        # 파일 다운로드
        files.download(results_path)

        print("Results saved and ready for download.")
    else:
        print("Error: No valid predictions or true labels for evaluation.")


Accuracy: 32.52%
F1 Score: 0.20
Confusion Matrix:
[[330   5   0  10]
 [201   6   0  21]
 [265   2   0  32]
 [233   3   0  36]]



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Results saved and ready for download.
